<a href="https://colab.research.google.com/github/chasubeen/python_selfstudy/blob/master/deep_pytorch/3.%20%EC%8B%AC%EC%B8%B5%20CNN%20%EC%95%84%ED%82%A4%ED%85%8D%EC%B2%98(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GoogLeNet과 Inception v3 살펴보기**

## **1) Inception 모듈**

In [2]:
import torch.nn as nn

In [3]:
class InceptionModule(nn.Module):
  def __init__(self, input_planes, n_channels1x1, n_channels3x3red, n_channels3x3, n_channels5x5red, n_channels5x5, pooling_planes):
    super(InceptionModule, self).__init__()
    # 1x1 합성곱 가지
    self.block1 = nn.Sequential(
      nn.Conv2d(input_planes, n_channels1x1, kernel_size = 1),
      nn.BatchNorm2d(n_channels1x1),
      nn.ReLU(True),
    )

    # 1x1 convolution -> 3x3 convolution branch
    self.block2 = nn.Sequential(
      nn.Conv2d(input_planes, n_channels3x3red, kernel_size = 1),
      nn.BatchNorm2d(n_channels3x3red),
      nn.ReLU(True),
      nn.Conv2d(n_channels3x3red, n_channels3x3, kernel_size = 3, padding = 1),
      nn.BatchNorm2d(n_channels3x3),
      nn.ReLU(True),
    )

    # 1x1 conv -> 5x5 conv branch
    self.block3 = nn.Sequential(
      nn.Conv2d(input_planes, n_channels5x5red, kernel_size = 1),
      nn.BatchNorm2d(n_channels5x5red),
      nn.ReLU(True),
      nn.Conv2d(n_channels5x5red, n_channels5x5, kernel_size=3, padding=1),
      nn.BatchNorm2d(n_channels5x5),
      nn.ReLU(True),
      nn.Conv2d(n_channels5x5, n_channels5x5, kernel_size=3, padding=1),
      nn.BatchNorm2d(n_channels5x5),
      nn.ReLU(True),
    )

    # 3x3 pool -> 1x1 conv branch
    self.block4 = nn.Sequential(
      nn.MaxPool2d(3, stride=1, padding=1),
      nn.Conv2d(input_planes, pooling_planes, kernel_size=1),
      nn.BatchNorm2d(pooling_planes),
      nn.ReLU(True),
    )

  def forward(self, ip):
    op1 = self.block1(ip)
    op2 = self.block2(ip)
    op3 = self.block3(ip)
    op4 = self.block4(ip)

    return torch.cat([op1,op2,op3,op4], 1)

## **2) Inception v1 모델**

In [4]:
class GoogLeNet(nn.Module):
    def __init__(self):
        super(GoogLeNet, self).__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(3, 192, kernel_size=3, padding=1),
            nn.BatchNorm2d(192),
            nn.ReLU(True),
        )

        self.im1 = InceptionModule(192,  64,  96, 128, 16, 32, 32)
        self.im2 = InceptionModule(256, 128, 128, 192, 32, 96, 64)

        self.max_pool = nn.MaxPool2d(3, stride=2, padding=1)

        self.im3 = InceptionModule(480, 192,  96, 208, 16,  48,  64)
        self.im4 = InceptionModule(512, 160, 112, 224, 24,  64,  64)
        self.im5 = InceptionModule(512, 128, 128, 256, 24,  64,  64)

        self.im6 = InceptionModule(512, 112, 144, 288, 32,  64,  64)
        self.im7 = InceptionModule(528, 256, 160, 320, 32, 128, 128)

        self.im8 = InceptionModule(832, 256, 160, 320, 32, 128, 128)
        self.im9 = InceptionModule(832, 384, 192, 384, 48, 128, 128)

        self.average_pool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(4096, 1000)

    def forward(self, ip):
        # ip: input
        op = self.stem(ip)

        out = self.im1(op)
        out = self.im2(op)

        op = self.maxpool(op)

        op = self.a4(op)

        op = self.b4(op)
        op = self.c4(op)
        op = self.d4(op)

        op = self.e4(op)

        op = self.max_pool(op)

        op = self.a5(op)
        op = self.b5(op)

        op = self.avgerage_pool(op)

        op = op.view(op.size(0), -1)

        op = self.fc(op)

        return op

## **3) 사전 훈련된 GoogLeNet 로딩**

In [5]:
import torchvision.models as models

model = models.googlenet(pretrained = True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:00<00:00, 160MB/s]


# **ResNet과 DenseNet 아키텍처**

## **1) ResNet의 잔차 블록 구현하기**

In [6]:
import torch.nn as nn

In [ ]:
class BasicBlock(nn.Module):

  multiplier=1

  def __init__(self, input_num_planes, num_planes, stride = 1):
    super(BasicBlock, self).__init__()
    self.conv_layer1 = nn.Conv2d(in_channels=input_num_planes, out_channels=num_planes,
                                 kernel_size=3, stride = stride,
                                 padding=1, bias=False)
    self.batch_norm1 = nn.BatchNorm2d(num_planes)
    self.conv_layer2 = nn.Conv2d(in_channels=num_planes, out_channels=num_planes,
                                 kernel_size=3, stride=1,
                                 padding=1, bias=False)
    self.batch_norm2 = nn.BatchNorm2d(num_planes)

    self.res_connnection = nn.Sequential()

    if stride > 1 or input_num_planes != self.multiplier*num_planes:
        self.res_connnection = nn.Sequential(
            nn.Conv2d(in_channels=input_num_planes, out_channels=self.multiplier*num_planes,
                      kernel_size=1, stride = stride, bias=False),
            nn.BatchNorm2d(self.multiplier*num_planes)
        )

  def forward(self, inp):
      op = F.relu(self.batch_norm1(self.conv_layer1(inp)))
      op = self.batch_norm2(self.conv_layer2(op))
      op += self.res_connnection(inp)
      op = F.relu(op)

      return op

## **2) DenseNet의 밀집 블록과 전환 블록 구현하기**

In [7]:
import torch.nn as nn

In [ ]:
### 밀집 블록

class DenseBlock(nn.Module):
    def __init__(self, input_num_planes, rate_inc):
        super(DenseBlock, self).__init__()
        self.batch_norm1 = nn.BatchNorm2d(input_num_planes)
        self.conv_layer1 = nn.Conv2d(in_channels=input_num_planes, out_channels=4*rate_inc, kernel_size=1, bias=False)
        self.batch_norm2 = nn.BatchNorm2d(4*rate_inc)
        self.conv_layer2 = nn.Conv2d(in_channels=4*rate_inc, out_channels=rate_inc, kernel_size=3, padding=1, bias=False)
    def forward(self, inp):
        op = self.conv_layer1(F.relu(self.batch_norm1(inp)))
        op = self.conv_layer2(F.relu(self.batch_norm2(op)))
        op = torch.cat([op,inp], 1)
        return op

In [ ]:
### 전환 블록

class TransBlock(nn.Module):
    def __init__(self, input_num_planes, output_num_planes):
        super(TransBlock, self).__init__()
        self.batch_norm = nn.BatchNorm2d(input_num_planes)
        self.conv_layer = nn.Conv2d(in_channels=input_num_planes, out_channels=output_num_planes, kernel_size=1, bias=False)
    def forward(self, inp):
        op = self.conv_layer(F.relu(self.batch_norm(inp)))
        op = F.avg_pool2d(op, 2)
        return op